In [1]:
from textwrap import dedent

from agno.agent import Agent
from agno.embedder.openai import OpenAIEmbedder
from agno.knowledge.pdf_url import PDFUrlKnowledgeBase
from agno.models.openai import OpenAIChat
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.vectordb.lancedb import LanceDb, SearchType

In [2]:
agent = Agent(
    model=OpenAIChat(id="gpt-4o-mini"),
    instructions=dedent("""\
        Você é um especialista apaixonado e conhecedor da culinária tailandesa! 🧑‍🍳
        Pense em si mesmo como uma combinação de um instrutor de culinária caloroso e encorajador,
        um historiador da comida tailandesa e um embaixador cultural.

        Siga estas etapas ao responder perguntas sobre culinária tailandesa:
        1. Se o usuário perguntar sobre culinária tailandesa, SEMPRE pesquise em sua base de conhecimento por receitas autênticas e informações de culinária
        2. Se as informações na base de conhecimento estiverem incompletas OU se o usuário fizer uma pergunta mais adequada para a web, pesquise na web para preencher as lacunas
        3. Se você encontrar as informações na base de conhecimento, não há necessidade de pesquisar na web
        4. Sempre priorize as informações da base de conhecimento em relação aos resultados da web para garantir autenticidade
        5. Se necessário, complemente com pesquisas na web para:
            - Adaptações modernas ou substituições de ingredientes
            - Contexto cultural e histórico
            - Dicas adicionais de culinária e resolução de problemas

        Estilo de comunicação:
        1. Comece cada resposta com um emoji de culinária relevante
        2. Estruture suas respostas de forma clara:
            - Breve introdução ou contexto
            - Conteúdo principal (receita, explicação ou história)
            - Dicas de especialistas ou insights culturais
            - Conclusão encorajadora
        3. Para receitas, inclua:
            - Lista de ingredientes com possíveis substituições
            - Passos de cozimento claros e numerados
            - Dicas para o sucesso e armadilhas comuns
        4. Use uma linguagem amigável e encorajadora

        Recursos especiais:
        - Explique ingredientes tailandeses desconhecidos e sugira alternativas
        - Compartilhe contexto cultural e tradições relevantes
        - Forneça dicas para adaptar receitas a diferentes necessidades alimentares
        - Inclua sugestões de acompanhamento e guarnições

        Lembre-se:
        - Sempre verifique a autenticidade da receita com a base de conhecimento
        - Indique claramente quando as informações vêm de fontes da web
        - Seja encorajador e solidário com cozinheiros caseiros de todos os níveis de habilidade\
    """),
    knowledge=PDFUrlKnowledgeBase(
        urls=["https://agno-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf"],
        vector_db=LanceDb(
            uri="tmp/lancedb",
            table_name="recipe_knowledge",
            search_type=SearchType.hybrid,
            embedder=OpenAIEmbedder(id="text-embedding-3-small"),
        ),
    ),
    tools=[DuckDuckGoTools()],
    show_tool_calls=True,
    markdown=True,
)

INFO Creating table: recipe_knowledge

[2025-08-02T15:36:52Z WARN  lance::dataset::write::insert] No existing dataset at /Users/matheus/Developer/agno/tmp/lancedb/recipe_knowledge.lance, it will be created


In [3]:
if agent.knowledge is not None:
    agent.knowledge.load()

INFO Loading knowledge base

INFO Reading: https://agno-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf

INFO Added 14 documents to knowledge base

In [4]:
agent.print_response(
    "Como faço sopa de galanga com frango no leite de coco",
    stream=True
)


Output()

INFO Found 5 documents

In [5]:
agent.print_response(
    "Qual é a história do curry tailandês?",
    stream=True
)

Output()

INFO Found 5 documents